# 👥 Customer Segmentation with RFM Analysis

This notebook performs **RFM (Recency, Frequency, Monetary)** analysis on transaction data to segment customers.

### What you'll do
- Load and clean the Online Retail dataset
- Engineer RFM features per customer
- Score R, F, M using quantiles
- Create segments (e.g., *Champions, Loyal, At Risk*)
- Visualize segment distribution and value

### Dataset
- Use the **UCI Online Retail** dataset (Excel): place the file at `data/Online Retail.xlsx`
- Columns used: `InvoiceNo, StockCode, Description, Quantity, InvoiceDate, UnitPrice, CustomerID, Country`

> Tip: If your file name differs, just update the path in the load cell below.

## 1) Setup & Load Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

%matplotlib inline

DATA_PATH = Path('data/Online Retail.xlsx')  # change if your filename is different
assert DATA_PATH.exists(), f"File not found: {DATA_PATH}. Place 'Online Retail.xlsx' under a 'data/' folder."

df = pd.read_excel(DATA_PATH)
print(df.shape)
df.head()

## 2) Quick Data Check

In [ ]:
df.info()

In [ ]:
df.describe(include='all').T.head(20)

## 3) Basic Cleaning
- Drop rows with missing `CustomerID`
- Remove cancellations/returns (negative quantities)
- Remove zero/negative prices
- Create `TotalPrice = Quantity * UnitPrice`
- Convert `InvoiceDate` to datetime

In [ ]:
raw = df.copy()
df = df.dropna(subset=['CustomerID']).copy()
df = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)].copy()
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']
df.head()

## 4) Compute RFM Features per Customer
- **Recency**: days since last purchase
- **Frequency**: number of invoices
- **Monetary**: total spend

In [ ]:
snapshot_date = df['InvoiceDate'].max() + pd.Timedelta(days=1)
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'nunique',
    'TotalPrice': 'sum'
}).rename(columns={'InvoiceDate': 'Recency', 'InvoiceNo': 'Frequency', 'TotalPrice': 'Monetary'})
rfm = rfm.astype({'Recency':'int', 'Frequency':'int'})
rfm.head()

## 5) RFM Scoring with Quantiles
- Lower **Recency** is better ⇒ invert when scoring
- Higher **Frequency**/**Monetary** is better

In [ ]:
def qcut_scores(series, q=5, ascending=True):
    quantiles = pd.qcut(series.rank(method='first'), q, labels=False) + 1
    if not ascending:
        quantiles = q + 1 - quantiles
    return quantiles

rfm['R_Score'] = qcut_scores(rfm['Recency'], q=5, ascending=False)
rfm['F_Score'] = qcut_scores(rfm['Frequency'], q=5, ascending=True)
rfm['M_Score'] = qcut_scores(rfm['Monetary'], q=5, ascending=True)
rfm['RFM_Score'] = rfm['R_Score'].astype(str) + rfm['F_Score'].astype(str) + rfm['M_Score'].astype(str)
rfm.head()

## 6) Map RFM Scores to Segments

In [ ]:
def segment_map(row):
    r, f, m = row['R_Score'], row['F_Score'], row['M_Score']
    if r >= 4 and f >= 4 and m >= 4:
        return 'Champions'
    if r >= 3 and f >= 4:
        return 'Loyal'
    if r >= 4 and f <= 2:
        return 'Potential Loyalist'
    if r <= 2 and f >= 4:
        return 'At Risk'
    if r <= 2 and f <= 2:
        return 'Hibernating'
    return 'Others'

rfm['Segment'] = rfm.apply(segment_map, axis=1)
rfm['Segment'].value_counts().head()

## 7) Visualize Segments

In [ ]:
seg_counts = rfm['Segment'].value_counts().sort_values(ascending=False)
plt.figure(figsize=(8,5))
seg_counts.plot(kind='bar')
plt.title('Customer Count by Segment')
plt.xlabel('Segment')
plt.ylabel('Customers')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
rev_by_seg = rfm.groupby('Segment')['Monetary'].sum().sort_values(ascending=False)
plt.figure(figsize=(8,5))
rev_by_seg.plot(kind='bar')
plt.title('Total Revenue by Segment')
plt.xlabel('Segment')
plt.ylabel('Revenue')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 8) Export Results

In [ ]:
Path('outputs').mkdir(exist_ok=True)
rfm.reset_index().to_csv('outputs/rfm_customers.csv', index=False)
print('Saved: outputs/rfm_customers.csv')

## 9) Notes for your README.md
- Explain what RFM is and why businesses use it
- Summarize key insights (e.g., *Champions are 12% of customers but 38% of revenue*)
- Mention tools used: **Python, pandas, matplotlib**
- Outline steps to reproduce: where to place the dataset and how to run the notebook